In [1]:
import os
%pwd

'/home/kumar/Documents/NEW JOB/UPSKILL/10 PERSONAL PROJECTS/RENAL-TUMOR-CLASSIFICATION/research'

In [2]:
os.chdir("../")#it goes one level up
!pwd

/home/kumar/Documents/NEW JOB/UPSKILL/10 PERSONAL PROJECTS/RENAL-TUMOR-CLASSIFICATION


In [3]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [4]:
from renal_tumor_detection.constants import *
from renal_tumor_detection.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
          

In [7]:
import os
import zipfile
import gdown
import urllib.request as request
from renal_tumor_detection import logger
from renal_tumor_detection.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        '''
        fetch data from given url
        '''
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file from the given path into data directory
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-25 14:47:04,991: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-25 14:47:04,994: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-25 14:47:04,995: INFO: common: created directory at: artifacts]
[2024-07-25 14:47:04,996: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-25 14:47:04,997: INFO: 1596745741: Downloading data from https://drive.google.com/file/d/1Mg6Gv7-sPl54NUL9Jx4h9_2dI943W4Ii/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Mg6Gv7-sPl54NUL9Jx4h9_2dI943W4Ii
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Mg6Gv7-sPl54NUL9Jx4h9_2dI943W4Ii&confirm=t&uuid=29c0f6d9-f3b9-4185-b1bf-f030a7fef789
To: /home/kumar/Documents/NEW JOB/UPSKILL/10 PERSONAL PROJECTS/RENAL-TUMOR-CLASSIFICATION/artifacts/data_ingestion/data.zip
100%|██████████| 46.4M/46.4M [00:01<00:00, 25.7MB/s]

[2024-07-25 14:47:11,801: INFO: 1596745741: Downloaded data from https://drive.google.com/file/d/1Mg6Gv7-sPl54NUL9Jx4h9_2dI943W4Ii/view?usp=sharing into file artifacts/data_ingestion/data.zip]
